# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('output_data/cities.csv')

vacation_df.head()

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Sao Mateus do Sul,0,BR,1580044480,46,-25.87,-50.38,75.33,0.94
1,1,Jamestown,99,US,1580044480,95,42.10,-79.24,33.01,10.00
2,2,Albany,53,US,1580044481,90,42.60,-73.97,35.01,4.38
3,3,San Policarpo,7,PH,1580044481,88,12.18,125.51,79.57,8.55
4,4,Punta Arenas,92,CL,1580044481,58,-53.15,-70.92,53.01,23.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Latitude", "Longitude"]].astype(float)


humidity = vacation_df["Humidity"].astype(float)



In [5]:
 # Create a poverty Heatmap layer
fig = gmaps.figure(zoom_level=2,center=(0,0))


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

not_too_hot = vacation_df

not_too_hot.drop(not_too_hot.loc[not_too_hot['Max Temperature'] > 80].index, inplace=True)

print(len(not_too_hot))


447


In [7]:
not_too_cool = not_too_hot

not_too_cool.drop(not_too_cool.loc[not_too_cool['Max Temperature'] < 72].index, inplace=True)

print(len(not_too_cool))

102


In [8]:
not_too_humid = not_too_cool

not_too_humid.drop(not_too_humid.loc[not_too_humid['Humidity'] > 50].index, inplace=True)

not_too_humid.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Sao Mateus do Sul,0,BR,1580044480,46,-25.87,-50.38,75.33,0.94
15,15,Cape Town,40,ZA,1580044324,40,-33.93,18.42,73.99,18.34
30,30,Sibi,0,PK,1580044485,32,29.54,67.88,72.54,7.83
44,44,Necochea,51,AR,1580044250,33,-38.55,-58.74,75.99,7.00
58,58,Busselton,41,AU,1580044489,35,-33.65,115.33,75.99,17.11
71,71,Puerto Ayora,93,EC,1580044491,22,-0.74,-90.35,75.00,1.01
101,101,Ponnampet,99,IN,1580044388,41,12.15,75.93,79.45,1.97
102,102,Flinders,17,AU,1580044497,22,-34.58,150.86,78.01,3.00
112,112,Port Elizabeth,90,ZA,1580044215,49,-33.92,25.57,73.40,35.57
238,238,Webuye,100,KE,1580044519,46,0.61,34.77,76.69,10.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = not_too_humid

hotel_df['Hotel Name']=""

hotel_df.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,0,Sao Mateus do Sul,0,BR,1580044480,46,-25.87,-50.38,75.33,0.94,
15,15,Cape Town,40,ZA,1580044324,40,-33.93,18.42,73.99,18.34,
30,30,Sibi,0,PK,1580044485,32,29.54,67.88,72.54,7.83,
44,44,Necochea,51,AR,1580044250,33,-38.55,-58.74,75.99,7.00,
58,58,Busselton,41,AU,1580044489,35,-33.65,115.33,75.99,17.11,
71,71,Puerto Ayora,93,EC,1580044491,22,-0.74,-90.35,75.00,1.01,
101,101,Ponnampet,99,IN,1580044388,41,12.15,75.93,79.45,1.97,
102,102,Flinders,17,AU,1580044497,22,-34.58,150.86,78.01,3.00,
112,112,Port Elizabeth,90,ZA,1580044215,49,-33.92,25.57,73.40,35.57,
238,238,Webuye,100,KE,1580044519,46,0.61,34.77,76.69,10.80,


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(hotel_name.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        #cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [11]:
hotel_df.head(15)


,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,0,Sao Mateus do Sul,0,BR,1580044480,46,-25.87,-50.38,75.33,0.94,Hotel Dom Leopoldo
15,15,Cape Town,40,ZA,1580044324,40,-33.93,18.42,73.99,18.34,Southern Sun Waterfront Cape Town
30,30,Sibi,0,PK,1580044485,32,29.54,67.88,72.54,7.83,Gorgaje Muhalla
44,44,Necochea,51,AR,1580044250,33,-38.55,-58.74,75.99,7.00,dyd -mar Hotel
58,58,Busselton,41,AU,1580044489,35,-33.65,115.33,75.99,17.11,Observatory Guest House
71,71,Puerto Ayora,93,EC,1580044491,22,-0.74,-90.35,75.00,1.01,Finch Bay Galapagos Hotel
101,101,Ponnampet,99,IN,1580044388,41,12.15,75.93,79.45,1.97,The Yellow Bamboo Resort & Spa
102,102,Flinders,17,AU,1580044497,22,-34.58,150.86,78.01,3.00,Shellharbour Resort & Conference Centre
112,112,Port Elizabeth,90,ZA,1580044215,49,-33.92,25.57,73.40,35.57,39 On Nile Guest House
238,238,Webuye,100,KE,1580044519,46,0.61,34.77,76.69,10.80,New Rays Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]  #narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))